# Zadania optymalizacyjne z elektrotechniki
## 3.1 Mostek rezystorowy

![challenge.png](obrazki/Rezystory1.png)

Na podstawie informacji zawartych w tabeli oraz na podstawie schematu należało określić rezystancję oporników tak aby wartości prądów znjdowały się w określonym zakresie a napięcie było zgodne z podanym, a moc rozpraszana na wyjściu była jak najmniejsza. 


W tym celu zainstalowano potrzebne bibloteki i przystąpiłem do tworzenia zadania optymalizacyjnego z wykorzystaniem Pulp.


In [1]:
!pip install pulp
!sudo apt-get install coinor-cbc glpk-utils coinor-clp

     |████████████████████████████████| 40.6MB 98kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5 libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev default-libmysqlclient-dev
The following NEW packages will be installed:
  coinor-cbc coinor-clp coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5 glpk-utils libamd2 libcolamd2
  libglpk40 libsuitesparseconfig5
0 upgraded, 12 newly installed, 0 to remove and 14 not upgraded.
Need to get 3,439 kB of archives.
After this operation, 9,832 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19.5 kB]
Get:3 h

In [6]:
# Import PuLP modeler functions
from pulp import *
import pandas as pd
prob = LpProblem("Mostek",LpMinimize)

I1 = LpVariable("Prąd1",0)
I2 = LpVariable("Prąd2",0)
I3 = LpVariable("Prąd3",0)
I4 = LpVariable("Prąd4",0)
I5 = LpVariable("Prąd5",0)


prob += 6*I1 + 10*I2 + 4*I3 + 7*I4 + 3*I5, "Moc"
prob += 3 <= I1, "Prąd1 min"
prob += I1 <= 5, "Prąd1 max"
prob += 1 <= I2, "Prąd2 min"
prob += I2 <= 3, "Prąd2 max"
prob += 1 <= I3, "Prąd3 min"
prob += I3 <= 3, "Prąd3 max"
prob += 1 <= I4, "Prąd4 min"
prob += I4 <= 3, "Prąd4 max"
prob += 3 <= I5, "Prąd5 min"
prob += I5 <= 5, "Prąd5 max"

prob.writeLP("MostekModel.lp")
prob.solve()

print ("Status:", LpStatus[prob.status])

for v in prob.variables():
    print (v.name, "=", v.varValue,"A,")

print("R1 =", 6/I1.varValue,"k,")
print("R2 =", 10/I2.varValue,"k,")
print("R3 =", 4/I3.varValue,"k,")
print("R4 =", 7/I4.varValue,"k,")
print("R5 =", 3/I5.varValue,"k")


Status: Optimal
Prąd1 = 3.0 A,
Prąd2 = 1.0 A,
Prąd3 = 1.0 A,
Prąd4 = 1.0 A,
Prąd5 = 3.0 A,
R1 = 2.0 k,
R2 = 10.0 k,
R3 = 4.0 k,
R4 = 7.0 k,
R5 = 1.0 k


Zmiennymi decyzyjnymi w tym przypadku zrobiłem prądy. W przypadku gdy mamy stałe napięcia to zmniejszając prąd zmniejszamy również wydzielaną moc. Pierwszym ograniczeniem była moc, która jest sumą mocy wydzielanych na rezystancji. Z racji, że napięcie jest stałe sprowadza się to do prostej sumy iloczynu napięć i prądów

Kolejnymi ograniczeniami są przedziały prądów jakie mogą płynąć przez rezystory.


Po wyznaczeniu prądów wyliczono wartości rezystancji.


##  3.2 Układ rezystorów

![challenge.png](obrazki/Rezystory2.png)

Na podstawie schematu i danych należało wyznaczyć maksymalny prąd, który może popłynąć przez gałąź. Określone mieliśmy rezystancje kolejnych rezystorów oraz ich maksymalne prądy.

In [10]:
# Import PuLP modeler functions
from pulp import *
import pandas as pd
prob = LpProblem("UkładRezystorów",LpMaximize)

I1 = LpVariable("Prąd1",0)
I2 = LpVariable("Prąd2",0)
I3 = LpVariable("Prąd3",0)
I4 = LpVariable("Prąd4",0)
I5 = LpVariable("Prąd5",0)

prob += I3, "Maksymalny prąd układu"
prob += I1 + I2 == I3, "Prąd lewa strona"
prob += I5 + I4 == I3, "Prąd prawa strona"
prob += I1*8 == I2*6, "Napięcie po lewej"
prob += I5*8 == I4*10, "Napięcie po prawej"
prob += I1 <= 2, "I1"
prob += I2 <= 3, "I2"
prob += I3 <= 4, "I3"
prob += I4 <= 2, "I4"
prob += I5 <= 2, "I5"

prob.writeLP("UkladModel.lp")
prob.solve()
print ("Status:", LpStatus[prob.status])

for v in prob.variables():
    print (v.name, "=", v.varValue, "A,")

print("Maksymalny prąd wynosi ",I3.varValue,"A")

Status: Optimal
Prąd1 = 1.5428571 A,
Prąd2 = 2.0571429 A,
Prąd3 = 3.6 A,
Prąd4 = 1.6 A,
Prąd5 = 2.0 A,
Maksymalny prąd wynosi  3.6 A


W tym wypadku maksymalny prąd dla całego układu narzuca nam rezystor R3. Następnie dodano ograniczenia wynikajace z I prawa Kirchhoffa, że suma prądów wpływająca do oczka musi być równa wypływającym. Rezystory połączone równolegle muszą mieć ten sam spadek napięcia. Finalnie wprowadzono ograniczenie wynikające z tabeli podanej w zadaniu.

Program poprawnie wyliczył wartość prądu. 



Ze względu na małą złożoność wykonanych operacji oraz fakt, że było niewiele zmiennych większość rzeczy wpisano po prostu w kod, bez tworzenia tablic i podobnych konstrukcji. 